# Requirements
Requirements for this tutorial:
1. Docker
2. Starting the docker container specified as part of this repository
```dos
docker compose - todo
```


# Description
In this tutorial, we will learn how to load data MRI images saved in the `.nii` format using medpy package. We will use the sample data that comes with the `bric_radiomics` package. After, we will compute the surface features using the `bric_radiomics` package.

The tutorial will take the following steps:
1. Importing the packages
1. Configuring the logging to view output generated by the bric_radiomics package
1. Converting `.nii` files to a numpy array and a metadata object
1. Converting numpy array to a sanitized mask using `bric_radiomics` utility functions
1. Compute the surface measures


# Importing the packages
```python
import logging
import bric_radiomics as br
from medpy.io import load
import numpy as np
```

The statement `import logging` will load python logging framework. By default, the logging level on all packages is set to warning. We will be configuring this to DEBUG so we can see all output from all packages. 

The statement `import bric_radiomics as br` will load the bric radiomics package. We will be aliasing this as `br`.

The statement `from medpy.io import load` will load a utility function that can open medical imaging files.

The statement `import numpy as np` will load the numpy library.

In [1]:
import logging
import bric_radiomics as br
from medpy.io import load
import numpy as np

# Configuring the logging framework

```python
FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)
```

The two statements above configure the logging framework to output all levels of logging for all packages. Ideally, we would like to restrict the logging level to individual packages. However, due to keeping this tutorial short, we will take the expedited route.

In [2]:
FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)

# Converting `.nii` files to a numpy array and a metadata object

```python
path_to_image = br.get_sample_nii_path()
image_data, image_header = load(path_to_image)
```

The line `path_to_image = br.get_sample_nii_path()` will return the path of the sample nii image.
The line `image_data, image_header = load(path_to_image)` will return a tuple, where the first element, `image_data`, is the image numpy array and the second element, `image_header`, is the metadata about the nii image.


The [metadata](http://loli.github.io/medpy/tutorial/metadata.html) contains information about the voxel spacing and the offset of each pixel.


In [3]:
path_to_image = br.get_sample_nii_path()
image_data, image_header = load(path_to_image)

In [4]:
labels = np.unique(image_data)
labels

array([0, 1, 2], dtype=int16)

## Analyzing the image voxels.

```python
>>> labels = np.unique(image_data)
>>> labels
    array([0, 1, 2], dtype=int16)
```
This lets you see how the MRI image was annotated. For this specific image, we can see that there are **three** labels: 0,1,2

# Converting numpy array to a sanitized mask using `bric_radiomics` utility functions

```
sanitized_mask = br.convert_volume_into_mask(image_data, merge_labels=[1,2])
```

The function `br.convert_volume_into_mask` expects 2 parameters. The first parmeter is the numpy image. The second parameter is for merging labels. In this specific instance, we would like to merge the annotations of 1 and 2 into a single label.

In [5]:
sanitized_mask = br.convert_volume_into_mask(image_data, merge_labels=[1,2])

2020-09-10 04:04:20,467 DEBUG convert_volume_into_mask  Found Labels: [0, 1, 2]
2020-09-10 04:04:20,489 DEBUG convert_volume_into_mask  Setting mask label 0 to 0
2020-09-10 04:04:20,559 DEBUG convert_volume_into_mask  Setting mask label 1 to 1
2020-09-10 04:04:20,583 DEBUG convert_volume_into_mask  Setting mask label 2 to 1


# Compute the surface measures

The statement `features = br.compute_morphology_features(sanitized_mask)` will return an object with several getters to the surface measures.
```python
measures = features.get_surface_measures()
measures.computed_curvedness()
measures.computed_shape_index()
measures.computed_sharpness()
measures.computed_total_curvature()
```

These surface measure can be used for other purposes.
Consult the documentation for get a more comprehensive overview of each measure.

In [6]:
features = br.compute_morphology_features(sanitized_mask)

2020-09-10 04:04:20,780 DEBUG compute_morphology_features  mask type is float64
2020-09-10 04:04:20,780 INFO compute_morphology_features  Starting Smoothing (iterations=1, sigma=3.0)
2020-09-10 04:04:20,781 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2020-09-10 04:04:21,019 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2020-09-10 04:04:21,019 INFO compute_morphology_features  Coverting volume into triangles using marching cubes (spacing=(1.0, 1.0, 1.0),method=MarchingCubesAlgorithm.Lewiner,step_size=1)
2020-09-10 04:04:21,181 INFO compute_morphology_features  Converting triangles into Mesh
2020-09-10 04:04:21,227 INFO compute_morphology_features  Computing Curvature
2020-09-10 04:04:21,336 DEBUG compute_morphology_features  Curvature:
 row_header|shape|min|max|mean|std
Gaussian Curvuture       |(28434,)|-1.4221e+00|+2.0160e+01|+3.2112e-03|+1.5686e-01
Mean Curvuture           |(28434,)|-1.2873e+01|+1.6581e+01|-3.3433e-02|+2.7606e-01


In [7]:
measures = features.get_surface_measures()
measures.computed_curvedness()
measures.computed_shape_index()
measures.computed_sharpness()
measures.computed_total_curvature()

array([0.38167939, 1.35329316, 0.48350463, ..., 0.4831879 , 0.        ,
       1.25008587])